1. Get crv emissions per pool.
2. Get fees claimed per pool.
3. divide the two to get swap revenue per crv emitted per gauge.

In [3]:
# subgraph querying modules
import datetime

import pandas as pd
import requests

In [4]:
# plotting tools
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import seaborn as sns
sns.set_style("whitegrid")

In [15]:
# brownie
import os
import sys
sys.path.append("../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network_and_connect

ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network_and_connect(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

# Pools and Gauges

Get dict of pools (keys) and gauge (value)

In [16]:
ethereum_registry_contract = init_contract("0x90E00ACe148ca3b23Ac1bC8C240C2a7Dd9c2d7f5")
ethereum_registry_contract.info()

  @title Curve Registry
  @author Curve.Fi



Get pools and gauges:

In [26]:
pool_count = ethereum_registry_contract.pool_count()
eth_pools_and_gauges = {}
for pool_id in range(pool_count):

    pool_addr = ethereum_registry_contract.pool_list(pool_id)
    gauge_addr = ethereum_registry_contract.get_gauges(pool_addr)
    lp_token_addr = ethereum_registry_contract.get_lp_token(pool_addr)

    eth_pools_and_gauges[pool_addr] = [gauge_addr[0][0], lp_token_addr]

eth_pools_and_gauges

{'0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7': ['0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A',
  '0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490'],
 '0xDeBF20617708857ebe4F679508E7b7863a8A8EeE': ['0xd662908ADA2Ea1916B3318327A97eB18aD588b5d',
  '0xFd2a8fA60Abd58Efe3EeE34dd494cD491dC14900'],
 '0xA96A65c051bF88B4095Ee1f2451C2A9d43F53Ae2': ['0x6d10ed2cF043E6fcf51A0e7b4C2Af3Fa06695707',
  '0xaA17A236F2bAdc98DDc0Cf999AbB47D47Fc0A6Cf'],
 '0x79a8C46DeA5aDa233ABaFFD40F3A0A2B1e5A4F27': ['0x69Fb7c45726cfE2baDeE8317005d3F94bE838840',
  '0x3B3Ac5386837Dc563660FB6a0937DFAa5924333B'],
 '0xA2B47E3D5c44877cca798226B7B8118F9BFb7A56': ['0x7ca5b0a2910B33e9759DC7dDB0413949071D7575',
  '0x845838DF265Dcd2c412A1Dc9e959c7d08537f8a2'],
 '0x0Ce6a5fF5217e38315f87032CF90686C96627CAA': ['0x90Bb609649E0451E5aD952683D64BD2d1f245840',
  '0x194eBd173F6cDacE046C53eACcE9B953F28411d1'],
 '0x4CA9b3063Ec5866A4B82E437059D2C43d1be596F': ['0x4c18E409Dc8619bFb6a1cB56D114C3f592E0aE79',
  '0xb19059ebb43466C323583928285a49f558E572Fd'],

# CRV emissions per pool:

Convex subgraph.

In [5]:
crv_emissions_subgraph = "https://api.thegraph.com/subgraphs/name/convex-community/crv-emissions"

In [30]:
gauge_addr = "0x64e3c23bfc40722d3b649844055f1d51c1ac041d"

In [31]:
def get_query(gauge_addr: str):

    return f'''
            {{
              dailyEmissionSnapshots(
                  where: {{gauge: "{gauge_addr}"}}
              ) {{
                    id
                    crvMinted
                    gauge
                    timestamp
              }}
            }}
            '''

In [35]:
print(get_query(gauge_addr))


            {
              dailyEmissionSnapshots(
                  where: {gauge: "0x64e3c23bfc40722d3b649844055f1d51c1ac041d"}
              ) {
                    id
                    crvMinted
                    gauge
                    timestamp
              }
            }
            


In [34]:
r = requests.post(crv_emissions_subgraph, json={'query': get_query(gauge_addr)})
proposal_data = dict(r.json())
proposal_data['data']

{'dailyEmissionSnapshots': [{'crvMinted': '2255604506145609220',
   'gauge': '0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'id': '1597276800-0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'timestamp': '1597276800'},
  {'crvMinted': '189445820646429517425',
   'gauge': '0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'id': '1597363200-0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'timestamp': '1597363200'},
  {'crvMinted': '298543859417264715645',
   'gauge': '0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'id': '1597449600-0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'timestamp': '1597449600'},
  {'crvMinted': '343162393326241531393',
   'gauge': '0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'id': '1597536000-0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'timestamp': '1597536000'},
  {'crvMinted': '604463371288425719806',
   'gauge': '0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'id': '1597622400-0x64e3c23bfc40722d3b649844055f1d51c1ac041d',
   'timestamp': '1597622400

In [38]:
data = pd.DataFrame(proposal_data['data']['dailyEmissionSnapshots']).drop(columns=['id'])
data

,crvMinted,gauge,timestamp
0,2255604506145609220,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1597276800
1,189445820646429517425,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1597363200
2,298543859417264715645,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1597449600
3,343162393326241531393,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1597536000
4,604463371288425719806,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1597622400
...,...,...,...
95,114713500471204368755036,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1605484800
96,45212887172888178440405,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1605571200
97,142416917867394568009779,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1605657600
98,26489664123098784214449,0x64e3c23bfc40722d3b649844055f1d51c1ac041d,1605744000


In [ ]:
emissions_data = pd.DataFrame()
for pool_addr, pool_details in eth_pools_and_gauges.items():

    gauge_addr = pool_details[0]
    lp_token_addr = pool_details[1]

    try:

        # get crv emissions:
        r = requests.post(crv_emissions_subgraph, json={'query': get_query(gauge_addr)})
        proposal_data = dict(r.json())
        if 'id' in pool_data.columns:
            pool_data = pd.DataFrame(
                proposal_data['data']['dailyEmissionSnapshots']
            ).drop(columns=['id'])
        pool_data.crvMinted = pool_data.crvMinted.astype(float) * 1e-18
        pool_data.timestamp = pool_data.timestamp.astype(float)
        # join df
        emissions_data = pd.concat([emissions_data, pool_data])

    except ValueError:

        print(pool_data)
        break

In [ ]:
emissions_data['measurement_time'] = pd.to_datetime(emissions_data.timestamp, unit='s')
emissions_data

# veCRV fees per pool per week: